In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
def request_backoff(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }
    
    time.sleep(random.uniform(3,4))
    max_retries = 6
    delay_base = 3 #initial delay
    
    for retry_count in range (max_retries):
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            return response
        elif response.status_code == 429: #rate limited
            retry_delay = delay_base * (2 ** retry_count) + random.uniform(0, 1)  # Exponential backoff with random factor
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        elif response.status_code == 400: #no more jobs
            break
            
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            
    print("Max retries reached. Aborting.")
    return None
        
            
            
    

In [3]:
title = "Python Developer"
location = "Toronto"

In [4]:
start = 0
id_list = []
success = True

while success:
    
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&position=1&pageNum=0&start={start}"
    
    try:
        response = request_backoff(list_url)
        list_data = response.text
        list_soup = BeautifulSoup(list_data, "html.parser")
        page_jobs = list_soup.find_all("li")

    except Exception as e:
        page_jobs = None
        
    if not page_jobs:
        
        print("No more jobs")        
        break #exit the loop if no jobs are found
    
    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        if base_card_div and 'data-entity-urn' in base_card_div.attrs:
            job_id = base_card_div.get('data-entity-urn').split(":")[3]
            print(job_id)
            id_list.append(job_id)
            start += 1
            
    print(f"new start: {start}")
        
        
    
    

Rate limit exceeded. Retrying in 3.3992196945504447 seconds...
3963991109
3916715909
3761549941
3764242184
3888466050
3908676161
3981315315
3989388868
3899604236
3988731995
new start: 10
3973948827
3981613044
3989453972
3988767582
3982254023
3978218431
3985583337
3983520560
3997641162
3972652267
new start: 20
Rate limit exceeded. Retrying in 3.9067076056637555 seconds...
3997641162
3972652267
3973163388
3979095916
3981814346
3995562340
3912147836
3985192464
3833866774
3835566820
new start: 30
Rate limit exceeded. Retrying in 3.869341449149474 seconds...
Rate limit exceeded. Retrying in 6.294489342648092 seconds...
Rate limit exceeded. Retrying in 12.966929637092967 seconds...
Rate limit exceeded. Retrying in 24.227147258543845 seconds...
3987646923
3985192464
3888461614
3945927713
3984113201
3969083879
3619294803
3976134239
3984135699
3973166630
new start: 40
Rate limit exceeded. Retrying in 3.73619125594012 seconds...
Rate limit exceeded. Retrying in 6.906880796704384 seconds...
39597

Rate limit exceeded. Retrying in 3.705350551967487 seconds...
Rate limit exceeded. Retrying in 6.221324689024614 seconds...
3977669180
3995426890
3989876691
3861235268
3960524894
3977514489
3983858339
3907410413
3939630656
3970460768
new start: 469
3995017451
3954909886
3980576040
3984161784
3961794941
3982978553
3954346962
3983963680
3890859305
3993602385
new start: 479
Rate limit exceeded. Retrying in 3.0735354031589557 seconds...
Rate limit exceeded. Retrying in 6.94307506128269 seconds...
3890859305
3993602385
3983429849
3997368154
3977479044
3657056228
3993611342
3924693738
3960530103
3974084695
new start: 489
3995988598
3733747837
3961922567
3962059371
3978678120
3963230428
3966995092
3991946750
3960161906
3994996999
new start: 499
Rate limit exceeded. Retrying in 3.919302968450274 seconds...
3994807121
3993790558
3955781077
3868486675
3986106998
3964480055
3989864405
3925728898
3982031013
3900930695
new start: 509
Rate limit exceeded. Retrying in 3.86480989429095 seconds...
No m

In [5]:
#remove duplicates
id_list = list(set(id_list))

In [6]:
len(id_list)

434

In [ ]:
job_list = []

for job_id in id_list:

    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    
    try:

        response = request_backoff(job_url)

        if response.status_code == 200:

            job_soup = BeautifulSoup(response.text, 'html.parser')

            job_post = {}  # Initialize 'job_post' as an empty dictionary before using it

            # Attempt to fetch company name
            try:
                job_post["company"] = job_soup.find("div", {"class": "top-card-layout__card"}).find("a").find("img").get('alt')
            except:
                job_post["company"] = None

            # Attempt to fetch job title
            try:
                job_post["job-title"] = job_soup.find("div", {"class": "top-card-layout__entity-info"}).find("a").text.strip()  # Corrected to use job_soup
            except:
                job_post["job-title"] = None

            # Attempt to fetch level
            try:
                job_post["level"] = job_soup.find("ul", {"class": "description__job-criteria-list"}).find("li").text.replace("Seniority level", "").strip()  # Corrected to use job_soup
            except:
                job_post["level"] = None

            try:
                job_post["description"] = job_soup.find("div", {"class": "decorated-job-posting__details"}).text.strip()
            except:
                job_post["description"] = None

            try:
                job_post["url"] = job_url
            except:
                job_post["url"] = None

            job_list.append(job_post)
            print(job_post)
            print(f"Successfully fetched data for job ID {job_id}")

        else:
            print(f"Failed to fetch data for job ID {job_id}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while fetching data for job ID {job_id}: {e}")

{'company': 'AddSource ', 'job-title': 'Java Full Stack Developer - Corporate Lending Technologies', 'level': 'Entry level', 'description': 'Skills And Responsibilities5+ years of experience in IT, with an emphasis on Cloud, Spring Boot, REST API and micro-service development, and proven ability in delivery web-based applications.Practical experience of the following technologies and platformsCloud Architecture Google Cloud, Microsoft Azure, AWS etc.Knowledge of Domain Driven Design, Event Driven Architecture, High Availability and other architecture patterns and principlesExperience of working with Enterprise Architecture Frameworks and their applicationExperience in delivering code using TDD, CI/CD, code analysis and coverage tools to support automated testing and monitoringSQL/NoSQL databases (SQL Server, MongoDB, Cassandra), REST/SOAP Protocols, Events (Kafka), OpenAI specification.Popular Java/JavaScript Frameworks, Runtimes and Components (e.g. React, Node JS, Java, Spring Boot e

Rate limit exceeded. Retrying in 3.602854573322721 seconds...
{'company': 'Virtusa', 'job-title': 'Java Full Stack developer', 'level': 'Associate', 'description': 'ResponsibilitiesJob Description:Required Skills: Hands on experience in Java-J2EE, Spring, Springboot, Rest APIs, ORM (Hibernate/JPA) development.Proficient in Java8, MicroservicesExperience in UI/Angular (Secondary skill)Working knowledge in RDBMS as Oracle and SQLsStrong knowledge of Design patterns/techniques, SOLID design principlesUnderstanding of standard SDLC and Agile processesGood experience in the coding standards and code qualityAbility to understand requirements and develop code individually.Strong writing, communication, time-management skills and positive attitudeKnowledge working in Agile development.Nice to have skills: Agile, JIRA, Bitbucket, GIT\n        \n\n\n        \n            Show more\n          \n\n          \n\n\n\n        \n            Show less\n          \n\n          \n\n \n\n\n\n\n           

{'company': 'CPP Investments | Investissements RPC', 'job-title': 'Ingénieur en chef, Programmation Python, Technologies et données', 'level': 'Mid-Senior level', 'description': "Description De L'entrepriseFaites votre marque au sein d’un organisme de placement dynamique d’envergure mondialeLorsque vous entreprenez une carrière à Investissements RPC, vous vous joignez à l’un des investisseurs institutionnels les plus respectés et dont la croissance est la plus rapide au monde. Riche d’un actif sous gestion dont la valeur dépasse les 500 milliards de dollars à l’heure actuelle, Investissements RPC est un organisme de gestion de placements professionnel qui investit les fonds du Régime de pensions du Canada (RPC) partout dans le monde afin d’en assurer la viabilité à long terme. La caisse du RPC devrait atteindre 3 000 milliards de dollars d’ici 2050. Investissements RPC investit dans toutes les grandes catégories d’actif, notamment des actions de sociétés cotées en bourse et de sociétés

{'company': 'Synechron', 'job-title': 'Python Developer', 'level': 'Mid-Senior level', 'description': 'Synechron provided pay range\n          \n\n\n\n        This range is provided by Synechron. Your actual pay will be based on your skills and experience — talk with your recruiter to learn more.\n      \n\nBase pay range\n\n      CA$110,000.00/yr - CA$115,000.00/yr\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\nWe areAt Synechron, we believe in the power of digital to transform businesses for the better. Our global consulting firm combines creativity and innovative technology to deliver industry-leading digital solutions. Synechron’s progressive technologies and optimization strategies span end-to-end Artificial Intelligence, Consulting, Digital, Cloud & DevOps, Data, and Software Engineering, servicing an array of noteworthy financial services and technology firms. Through research and development initiatives in our FinLabs we develop solutions for modernization, from Artificial Intelligence and B

Rate limit exceeded. Retrying in 3.1919098488222306 seconds...
{'company': 'Stripe', 'job-title': 'Backend Engineer, Optimized Checkout & Link Data Engineering', 'level': 'Mid-Senior level', 'description': 'Who we areAbout StripeStripe is a financial infrastructure platform for businesses. Millions of companies—from the world’s largest enterprises to the most ambitious startups—use Stripe to accept payments, grow their revenue, and accelerate new business opportunities. Our mission is to increase the GDP of the internet, and we have a staggering amount of work ahead. That means you have an unprecedented opportunity to put the global economy within everyone’s reach while doing the most important work of your career.About The TeamThe Optimized Checkout & Link team at Stripe builds best-in-class checkout experiences across web and mobile that delight consumers and streamline checkout flows for merchants. Based across North America, we\'re a diverse team who are deeply passionate about red

...

Rate limit exceeded. Retrying in 3.1504961432223526 seconds...
{'company': 'Iris Software Inc.', 'job-title': 'Python/ETL Developer', 'level': 'Mid-Senior level', 'description': "Direct message the job poster from Iris Software Inc.\n\n\n\n              \n          \n        Amarpreet Singh\n\n\n\n\n\n\n\n            \n        Amarpreet Singh\n\n\n\n\n              \n        Talks about #banking, #careers, #jobalerts, #bankingjobs, and #bankingcareers\n      \n            \n\n \n\n\n\n\n\n\n\n\n\n\nHello,Iris Software's direct banking client looking to hire Python Developer based out of Toronto, ON.Kindly share your profile at amarpreet.singh@irissoftware.com.Job Title: Lead Python/ETL DeveloperLocation: Toronto, ONDuration: 18 – 24 + Months (Open ended SoW)Must have:Need a strong Python developer to lead a team of 5 offshore developers to develop Python based UI and cloud based ETL processes.Regards,Amarpreet Singh 200 Bay Str. Toronto, ON, M5H4E9200 Metroplex Drive, Suite #300 Edison

{'company': 'Zortech Solutions', 'job-title': 'Java Full Stack Developer', 'level': 'Entry level', 'description': 'Role: Java Full Stack Developer Location: Toronto, ON Position Type: Full Time """Skills and Responsibilities: Strong experience in Anuglar 7 and Material Design Framework,HTML,CSS 3 and JavaScript  Strong experience in Enterprise applications Development for various projects with expertise in Design, Development, and Integration of multi-tier architecture projects by using the strong technical skills on Core Java, Spring MVC,Spring Rest, Spring Batch, Spring Security, Spring Boot, EJB3.0, Web services, Containerized Platform, JPA. Good Knowledge on WebSphere Liberty profile  Solid knowledge of Agile development practices and strong working experience in DevOps tools like Devops, GitHub, Bit Bucket, JIRA, Jenkins, Ansible Docker, Open Shift, Confluence Experience in Hibernate, Struts, AngularJS, HTML5, CSS., JMS/MQ configuration. Strong Experience on the Security vulnerabi

{'company': 'CIBC', 'job-title': 'Full stack Developer (. Net, C#, Azure, SQL and Angular)', 'level': 'Entry level', 'description': "We’re building a relationship-oriented bank for the modern world. We need talented, passionate professionals who are dedicated to doing what’s right for our clients.At CIBC, we embrace your strengths and your ambitions, so you are empowered at work. Our team members have what they need to make a meaningful impact and are truly valued for who they are and what they contribute.To learn more about CIBC, please visit CIBC.comWhat You’ll Be DoingBy joining CIBC’s Capital Markets Treasury Applications Technology team, you'll be at the heart of CIBC’s digital transformation, building easy, flexible, and personalized banking solutions to enhance client experience and change the way that people bank. As a Full stack Developer (. Net, C#, Azure, SQL and Angular), you’ll actively participate in developing, testing and supporting the implementation of cross-functiona

Rate limit exceeded. Retrying in 3.490760249764306 seconds...
Rate limit exceeded. Retrying in 6.61934972876315 seconds...
Rate limit exceeded. Retrying in 12.404280276061758 seconds...
Rate limit exceeded. Retrying in 24.048646113784358 seconds...
{'company': 'Blue J', 'job-title': 'Backend Developer', 'level': 'Not Applicable', 'description': "Blue J provided pay range\n          \n\n\n\n        This range is provided by Blue J. Your actual pay will be based on your skills and experience — talk with your recruiter to learn more.\n      \n\nBase pay range\n\n      CA$125,000.00/yr - CA$145,000.00/yr\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n          About Blue J Blue J’s ambition is to be the world leader in generative AI for tax experts. Our AI tax research software is widely regarded as the best in the market. We are racing ahead with an exciting product development roadmap to continue to delight our customers and to ensure that our users can generate the very best possible tax research ans

Rate limit exceeded. Retrying in 3.403541460346375 seconds...
{'company': 'Cockroach Labs', 'job-title': 'Software Engineer, Backend (Migrations) - Toronto', 'level': 'Entry level', 'description': "Databases are the beating heart of every business in the world.Cockroach Labs is the team behind CockroachDB, the most highly evolved cloud-native, distributed SQL database on the planet. We created CockroachDB and CockroachDB Cloud to deliver the ability to build and scale apps with fewer obstacles, more freedom, and greater efficiency. Today, Cockroach Labs helps companies of all sizes—and the apps they develop—scale fast, survive disaster, and thrive everywhere. Join us on our mission to make data easy.About The RoleAt Cockroach Labs, we strive to deliver a delightful experience to our customers starting from their first interaction with our database. Creating such experiences that make data easy and painless is paramount to our mission. In particular, we are looking for a senior software

In [8]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,company,job-title,level,description,url
0,Epsilon Solutions Ltd.,Python Developer,Entry level,"Role: Python DeveloperJob Description"" 5+ year...",https://www.linkedin.com/jobs-guest/jobs/api/j...
1,Zortech Solutions,"Data Engineer-SQL, Python, ETL- Canada",Entry level,"Role: Data Engineer-SQL, Python, ETLLocation: ...",https://www.linkedin.com/jobs-guest/jobs/api/j...
2,Boomerang,Software Engineer (Full-Stack),Employment type\n \n\n Full-time,Direct message the job poster from Boomerang\n...,https://www.linkedin.com/jobs-guest/jobs/api/j...
3,Phoenix (YC W21),Full-Stack Developer,Entry level,What we're looking forWe are looking for a ful...,https://www.linkedin.com/jobs-guest/jobs/api/j...
4,Scotiabank,Senior Software Engineer | Python Developer,Not Applicable,Requisition ID: 189673Join a purpose driven wi...,https://www.linkedin.com/jobs-guest/jobs/api/j...
...,...,...,...,...,...
108,Zortech Solutions,Python Developer,Entry level,Role: Python DeveloperLocation: Toronto/Remote...,https://www.linkedin.com/jobs-guest/jobs/api/j...
109,Seven Hills Group Technologies Inc.,Python Developer with AWS.,Entry level,"Hi,I am Santhi swaroop, With Seven Hills Group...",https://www.linkedin.com/jobs-guest/jobs/api/j...
110,ClifyX,Full Stack Developer,Entry level,Reg ID : 9253432 Role/Skill : Full Stack Devel...,https://www.linkedin.com/jobs-guest/jobs/api/j...
111,Della Connect,Backend Developer,Entry level,Our company is a rapidly growing technology fi...,https://www.linkedin.com/jobs-guest/jobs/api/j...


In [9]:
jobs_df.to_csv('Toronto_Python_Developer.csv', index=False)